# Problem statement

Kyphosis is an abnormally excessive convex curvature of the spine. The kyphosis data frame has 81 rows and 4 columns. representing data on children who have had corrective spinal surgery. Dataset contains 3 inputs and 1 output

INPUTS:

Age: in months

Number: the number of vertebrae involved

Start: the number of the first (topmost) vertebra operated on.

OUTPUTS:

Kyphosis: a factor with levels absent present indicating if a kyphosis (a type of deformation) was present after the operation.

# Step 1: Data reading and insight

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('whitegrid')
plt.style.use('seaborn-deep')

import warnings
warnings.filterwarnings('ignore')
import sklearn.metrics as skm
import sklearn.model_selection as skms
import sklearn.preprocessing as skp
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import random


seed = 12
np.random.seed(seed)

from datetime import date

In [2]:
# read the data using pandas dataframe
kyphosis = pd.read_csv('../input/kyphosis-dataset/kyphosis.csv')


kyphosis.head(5)

In [3]:
# important funtions
def datasetShape(df):
    rows, cols = df.shape
    print("The dataframe has",rows,"rows and",cols,"columns.")
    
# select numerical and categorical features
def divideFeatures(df):
    numerical_features = df.select_dtypes(include=[np.number])
    categorical_features = df.select_dtypes(include=[np.object]).drop('Kyphosis', axis=1)
    return numerical_features, categorical_features

In [4]:
kyphosis.info()

In [5]:
kyphosis.describe()

# Step 2: EDA

In [6]:
# check null values

pd.DataFrame(kyphosis.isnull().sum(), columns=["Null Count"]).style.background_gradient(cmap='Blues')

In [7]:
numerical_features, categorical_features = divideFeatures(kyphosis)

In [8]:
LabelEncoder_y = LabelEncoder()
kyphosis['Kyphosis'] = LabelEncoder_y.fit_transform(kyphosis['Kyphosis'])

In [9]:
# boxplots of numerical features for outlier detection

fig = plt.figure(figsize=(16,30))
for i in range(len(numerical_features.columns)):
    fig.add_subplot(9, 5, i+1)
    sns.boxplot(y=numerical_features.iloc[:,i])
plt.tight_layout()
plt.show()

In [10]:
sns.color_palette("Blues", as_cmap=True)
GnBu_palette = sns.color_palette("GnBu",10)
Blues_palette = sns.color_palette("Blues",10)
sns.palplot(Blues_palette)
sns.palplot(GnBu_palette)

In [11]:
fig, axes = plt.subplots(1,2,figsize=(10,4))

kyphosis['Kyphosis'].value_counts().plot.pie(
    explode=[0,0.1],autopct='%1.1f%%',ax=axes[0],shadow=True, colors=[Blues_palette[1],Blues_palette[3]]
)

sns.countplot('Kyphosis',data=kyphosis,ax=axes[1], palette=[GnBu_palette[6],GnBu_palette[7]])
axes[1].patch.set_alpha(0)

fig.text(0.28,0.92,"Distribution of Outcome percent", fontweight="bold", fontfamily='serif', fontsize=17)

plt.show()

**Unbalanced classes** (solve with SMOTE function)

In [13]:
# Distribution of discrete data over output column
discrete_features = numerical_features.select_dtypes(include=['integer'])

fig, axes = plt.subplots(1,2, figsize=(14,6) )
#fig, ax =plt.subplots(1,2)
sns.countplot(x = 'Number', hue = 'Kyphosis', data=kyphosis, ax=axes[0])
sns.countplot(x = 'Start', hue = 'Kyphosis', data=kyphosis, ax=axes[1])
fig.show()

- Vertebrae number 1 and 9 are the only ones with no cases of kyphosis
- Vertebrae number 7 and 10 show more cases of kyphosis than non-kyphosis cases
- The start vertebrae numbers 5, 6, 8 and 12 have more cases of scyphosis than non-syphosis cases

In [14]:
sns.pairplot(kyphosis, hue = 'Kyphosis', vars = ['Age'],  height=8 )

- between 100 and 150 months of age we see the highest number of cases of kyphosis

In [15]:
kyphosis[kyphosis.columns[:5]].corr().style.background_gradient(cmap='Blues')

- There is a rather moderate negative correlation between 'start' and "Kyphosis"

# Step 3: Data Cleaning

In [87]:
# No data cleaning required!!

# Step 4: Data Preparation

In [16]:
# extract all skewed features
skewed_features = numerical_features.apply(lambda x: x.skew()).sort_values(ascending=False)

In [17]:
# transform skewed features
for feat in skewed_features.index:
    if skewed_features.loc[feat] > 0.5:
        kyphosis[feat] = np.log1p(kyphosis[feat])

# Step 5: Modelling

Split Train-Test Data

In [18]:
# shuffle samples
df_shuffle = kyphosis.sample(frac=1, random_state=seed).reset_index(drop=True)

In [19]:
df_y = df_shuffle.pop('Kyphosis')
df_X = df_shuffle

# split into train dev and test
X_train, X_test, y_train, y_test = skms.train_test_split(df_X, df_y, train_size=0.7, random_state=seed)
print(f"Train set has {X_train.shape[0]} records out of {len(df_shuffle)} which is {round(X_train.shape[0]/len(df_shuffle)*100)}%")
print(f"Test set has {X_test.shape[0]} records out of {len(df_shuffle)} which is {round(X_test.shape[0]/len(df_shuffle)*100)}%")

Feature Scaling

In [21]:
scaler = skp.StandardScaler()

# apply scaling to all numerical variables (float64) except dummy variables as they are already between 0 and 1

X_train[numerical_features.columns] = scaler.fit_transform(X_train[numerical_features.columns])

# scale test data with transform()
X_test[numerical_features.columns] = scaler.transform(X_test[numerical_features.columns])


# view sample data
X_train.describe()

In [22]:
smote = SMOTE()
X_over, y_over = smote.fit_resample(X_train,y_train)

In [23]:
fig, axes = plt.subplots(1,2,figsize=(10,4))

y_over.value_counts().plot.pie(
    explode=[0,0.1],autopct='%1.1f%%',ax=axes[0],shadow=True, colors=[Blues_palette[1],Blues_palette[3]]
)

sns.countplot(y_over, ax=axes[1], palette=[GnBu_palette[6],GnBu_palette[7]])
axes[1].patch.set_alpha(0)

fig.text(0.28,0.92,"Distribution of After  SMOTE Output percent", fontweight="bold", fontfamily='serif', fontsize=17)

plt.show()

**Before SMOTE**

In [26]:
# Evaluation function definition

def get_clf_eval(y_test, pred = None, pred_proba = None):
    confusion = confusion_matrix(y_test, pred)
    accuacy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('confusion')
    print(confusion)
    print('accuracy : {}'.format(np.around(accuacy,4)))
    print('precision: {}'.format(np.around(precision,4)))
    print('recall : {}'.format(np.around(recall,4)))
    print('F1 : {}'.format(np.around(f1,4)))  
    print('ROC_AUC : {}'.format(np.around(roc_auc,4)))

Logistic Regression Model

In [27]:
lg_reg = LogisticRegression()

lg_reg.fit(X_train, y_train)
pred = lg_reg.predict(X_test)
pred_proba = lg_reg.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

Random Forest Model

In [28]:
rf_clf = RandomForestClassifier()
param = {'n_estimators' : [100],
         'max_depth':[8,9,10],
         'min_samples_split':[2,5,7],
         'min_samples_leaf':[6.5,7,7.5]
        }

In [29]:
grid = GridSearchCV(rf_clf,param_grid = param,scoring = 'accuracy',cv=5)
grid.fit(X_train ,y_train)

In [30]:
grid.best_params_

In [31]:
grid.best_score_

In [32]:
pred = grid.predict(X_test)
pred_proba = grid.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

LightGBM Classification Modeling

In [33]:
model = lgb.LGBMClassifier(
    n_estimators=400,
    num_leaves=20,
    min_data_in_leaf=60,
    learning_rate=0.01,
    boosting='gbdt',
    objective='binary',
    metric='auc',
    Is_training_metric=True,
    n_jobs=-1
)

In [34]:
model.fit(X_train,y_train)

In [35]:
pred = model.predict(X_test)
pred_proba = model.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

SGD Classification Model

In [36]:
from sklearn.linear_model import SGDClassifier

clf_SGD = SGDClassifier(loss="squared_loss", penalty="l2", max_iter=4500,tol=-1000, random_state=1)
clf_SGD.fit(X_train,y_train)
pred = clf_SGD.predict(X_test)
pred_proba = lg_reg.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

**After SMOTE**

Logistic Regression Model

In [37]:
lg_reg = LogisticRegression()

lg_reg.fit(X_over, y_over)
pred = lg_reg.predict(X_test)
pred_proba = lg_reg.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

Random Forest Model

In [38]:
rf_clf = RandomForestClassifier()
param = {'n_estimators' : [200],
         'max_depth':[10],
         'min_samples_split':[2],
         'min_samples_leaf':[7]
        }

In [39]:
grid = GridSearchCV(rf_clf,param_grid = param,scoring = 'accuracy',cv=5)
grid.fit(X_over ,y_over)

pred = grid.predict(X_test)
pred_proba = grid.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

LightGBM Classification Model

In [40]:
model = lgb.LGBMClassifier(
    n_estimators=400,
    num_leaves=20,
    min_data_in_leaf=60,
    learning_rate=0.01,
    boosting='gbdt',
    objective='binary',
    metric='auc',
    Is_training_metric=True,
    n_jobs=-1
)

In [41]:
model.fit(X_over,y_over)

In [42]:
pred = model.predict(X_test)
pred_proba = model.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

SGD Classification Model

In [43]:
from sklearn.linear_model import SGDClassifier

clf_SGD = SGDClassifier(loss="squared_loss", penalty="l2", max_iter=4500,tol=-1000, random_state=1)
clf_SGD.fit(X_over,y_over)
pred = clf_SGD.predict(X_test)
pred_proba = lg_reg.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

# Final considerations

The RF model before SMOTE obtained the highest accuracy value (96%)